<a href="https://colab.research.google.com/github/orico/KernelAlgorithms/blob/master/Time_Complexity_Benchmarking_for_Kernel_%26_Non_Kernel_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import time
from scipy.stats import norm, multivariate_normal, matrix_normal, gaussian_kde
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.utils import check_random_state
from sklearn.datasets import load_digits, fetch_mldata, fetch_california_housing, load_breast_cancer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans 
from sklearn.svm import LinearSVC, SVC 
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split 
from sklearn.neighbors.kde import KernelDensity
from sklearn.mixture import GaussianMixture

import gc 

!pip install memory_profiler
%load_ext memory_profiler
!pip install cython 
!pip install tslearn
from tslearn.clustering import GlobalAlignmentKernelKMeans

We start by downloading our data and splitting it to train and test, according to known MNIST definitions 60K/10K split. later the train-set will be split to train and validation.


In [2]:
class Normalize(object):
    
    def normalize(self, X_train, X_test):
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_test  = self.scaler.transform(X_test)
        return (X_train, X_test)
    
    def inverse(self, X_train, X_test):
        X_train = self.scaler.inverse_transform(X_train)
        X_test  = self.scaler.inverse_transform(X_test)
        return (X_train, X_test) 
       
data = load_breast_cancer() #fetch_mldata('MNIST original')
X = data.data.astype('float64')
y = data.target

print ('Data:', X.shape, y.shape)

#split train/val/test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, np.unique(y_train))
print(X_test.shape, np.unique(y_test))

norm_ = Normalize()
X_train, X_test = norm_.normalize(X_train, X_test)

Data: (569, 30) (569,)
(512, 30) [0 1]
(57, 30) [0 1]


In [0]:
def calcAcc(clf):
  classif_rate = np.mean(clf.test_y_predicted.ravel() == y_test.ravel()) * 100
  print("Accuracy rate for %f " % (classif_rate))    
  print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, clf.test_y_predicted))
  
class BaseModel(object):

    def __init__(self):
        pass

    def fit(self):
        pass
      
    def predict_or_transform(self):
        pass
      
      
class SvmModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = SVC(C=1, kernel='linear', probability=True, class_weight=c_weight)
        self.model.fit(X_train, y_train)
        
    def predict_or_transform(self, X_test):
        self.test_y_predicted = self.model.predict(X_test) 

        
class LinearSvmModell2(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = clf = LinearSVC(random_state=42, class_weight=c_weight)
        self.model.fit(X_train, y_train)
        
    def predict_or_transform(self, X_test):
        self.test_y_predicted = self.model.predict(X_test) 
        
        
class LinearSvmModell1(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = clf = LinearSVC(random_state=42, penalty = 'l1', class_weight=c_weight)
        self.model.fit(X_train, y_train)
        
    def predict_or_transform(self, X_test):
        self.test_y_predicted = self.model.predict(X_test) 
        
        
class PCAModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = PCA()
        self.model.fit(X_train)     
      
    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.transform(X_test)
        
        
class KernelPCAModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = KernelPCA(kernel = 'linear')
        self.model.fit(X_train)          

    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.transform(X_test)
        
        
class KernelKmeansModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = GlobalAlignmentKernelKMeans(n_clusters=2, verbose = 0)
        self.model.fit(X_train)         

    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.predict(X_test)
        
        
class KmeansModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = KMeans(n_clusters=2)
        self.model.fit(X_train)           

    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.predict(X_test)        
      
        
class KDEModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = KernelDensity()
        self.model.fit(X_train)           

    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.score_samples(X_test)        


class GaussianMixtureModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):
        self.model = GaussianMixture(n_components=2)
        self.model.fit(X_train)           
    
    def predict_or_transform(self, X_test):
        self.X_test_trans = self.model.transform(X_test)         
        
        
class PDFModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):  
        self.model = norm.pdf(X_train)
    
    def predict_or_transform(self, X_test):
        pass
#         self.X_test_trans = self.model.transform(X_test)                 

class GaussianKdeModel(BaseModel):
  
    def fit(self, X_train, y_train, c_weight):  
        self.model = gaussian_kde(X_train.T)
    
    def predict_or_transform(self, X_test): 
        self.X_test_trans = self.model.evaluate(X_test.T)
        
def profiling_fit(clf):
  clf.fit(X_train, y_train, 'balanced')

In [4]:
print("PDF vs KDE vs Gaussian KDE")
# gmx_clf = GaussianMixtureModel()
# %timeit -n 1 profiling_fit(gmx_clf) 

pdf_clf = PDFModel()
%timeit -n 10 profiling_fit(pdf_clf)

GKde_clf = GaussianKdeModel()
%timeit -n 10 profiling_fit(GKde_clf)

kde_clf = KDEModel()
%timeit -n 10 profiling_fit(kde_clf)
print()

PDF vs KDE vs Gaussian KDE
10 loops, best of 3: 822 µs per loop
10 loops, best of 3: 581 µs per loop
10 loops, best of 3: 399 µs per loop



In [5]:
print("LinearSVM vs Linear Kernel-SVM")
linearsvm_clf = LinearSvmModell2()
%timeit -n 10 profiling_fit(linearsvm_clf)
svm_clf = SvmModel()
%timeit -n 10 profiling_fit(svm_clf)
print()

print("PCA vs Linear Kernel-PCA")
pca_clf = PCAModel()
%timeit -n 10 profiling_fit(pca_clf)
kpca_clf = KernelPCAModel()
%timeit -n 10 profiling_fit(kpca_clf)
print()

print("Kmeans vs Kernel-KMeans")
kmeans_clf = KmeansModel()
%timeit -n 10 profiling_fit(kmeans_clf) 
KKmeans_clf = KernelKmeansModel()
%timeit -n 10 profiling_fit(KKmeans_clf)
print()

LinearSVM vs Linear Kernel-SVM
10 loops, best of 3: 4.52 ms per loop
10 loops, best of 3: 11.9 ms per loop

PCA vs Linear Kernel-PCA
10 loops, best of 3: 1.39 ms per loop
10 loops, best of 3: 42.7 ms per loop

Kmeans vs Kernel-KMeans
10 loops, best of 3: 25.6 ms per loop
10 loops, best of 3: 12.2 s per loop



In [6]:
def profiling_pred_trans(clf):
  clf.predict_or_transform(X_test) 

print("KDE vs Gaussian KDE")
# %timeit -n 10 profiling_pred_trans(pdf_clf)
%timeit -n 10 profiling_pred_trans(kde_clf)
%timeit -n 10 profiling_pred_trans(GKde_clf) 
print()
  
print("LinearSVM vs Linear Kernel-SVM")
%timeit -n 10 profiling_pred_trans(linearsvm_clf)
%timeit -n 10 profiling_pred_trans(svm_clf)
print()

print("PCA vs Linear Kernel-PCA")
%timeit -n 10 profiling_pred_trans(pca_clf)
%timeit -n 10 profiling_pred_trans(kpca_clf)
print()

print("Kmeans vs Kernel-KMeans")
%timeit -n 10 profiling_pred_trans(kmeans_clf) 
%timeit -n 10 profiling_pred_trans(KKmeans_clf)
print()

KDE vs Gaussian KDE
10 loops, best of 3: 3.98 ms per loop
10 loops, best of 3: 3.9 ms per loop

LinearSVM vs Linear Kernel-SVM
10 loops, best of 3: 56.2 µs per loop
10 loops, best of 3: 321 µs per loop

PCA vs Linear Kernel-PCA
10 loops, best of 3: 59.6 µs per loop
10 loops, best of 3: 1.93 ms per loop

Kmeans vs Kernel-KMeans
10 loops, best of 3: 366 µs per loop
10 loops, best of 3: 2.78 s per loop



[The](https://www.quora.com/Whats-the-difference-between-LibSVM-and-LibLinear) QP solver used in libSVM is targeted to work for both linear and non-linear kernel. The training time complexity is somewhere around O(n2) to O(n3). The solver in libLinear is targeted to primarily work with linear kernel and on top of that it offers several variations (L1/L2 regularization, L1/L2 loss etc.). The training time complexity is somewhere around O(n).